In [1]:
# Imports
import numpy as np
import pandas as pd
from num2words import num2words  # Converting numbers to word form
import ijson  # Parser for large json files
import json

import nltk  # Import nltk module
from nltk.corpus import stopwords  # Stop word dictionary
from nltk.stem import PorterStemmer  # Stems words
from nltk.tokenize import word_tokenize  # Tokenizer

from textblob import TextBlob

import sqlite3 # Import sqlite to store data at end to sql database

import os # For obtaining local directory to store file

# Download stopwords dictionary
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/eitan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
file_name = "yelp_academic_dataset_review.json"

# Example of a json object in the review file:
# {"review_id":"btHrA_nXUceLqZRvymvXng","user_id":"amaOELCfgLup2MwE2j8PfA","business_id":"_v3DcLatG70adfYzWTd-CQ","stars":5.0,"useful":2,"funny":2,"cool":1,"text":"I love this store!","date":"2015-03-18 21:09:07"}

# Order of columns:
#"review_id", "user_id", "business_id", "stars","useful","funny","cool","text","date"

In [3]:
# Word cleaning/preprocessing function
def text_preprocess(text):
    '''
    Function: Process takes an incoming JSON text review and preprocesses it by:
    1. converting string to lowercase
    2. removing stop words (is, it, a, but etc)
    3. converts numbers to their word form (1 -> one) NOTE: Unclear if this is neccessary, but was suggsted online
    4. Stem the words (convert words like programming -> program)
    
    Input: JSON item "review"
    Output: preprocessed string
    '''
    # Define array to store processed words:
    processed_words = []

    # Define stop_words dict using nltk package
    stop_words = set(stopwords.words('english'))

    # Correct the string with TextBlob spell checker

    blob = TextBlob(text)
    text = str(blob.correct())

    # Split string into list
    split = text.split()
    
    # Create stemmer object
    ps = PorterStemmer()

    # Iterate through each word
    for word in split:
        # Convert word to lowercase
        word = word.lower()

        # Check if word is a stop word before proceeding
        if word in stop_words:
            continue # Continue will skip current iteration (current word) if it is a stop word
        
        if word == 'infinity': # Edgecase check if word is infinity, as num2words does not handle it correctly
            processed_words.append('infinity')
            continue # Continue to next word

        # Convert numbers to word form 
        try:
            # Check if word is a number
            float(word)  # This will raise ValueError if the word is not a number
            word = num2words(word)
        except ValueError:
            # If it's not a number, we can stem the word
            word = ps.stem(word)
        
        processed_words.append(word)

    # Join the processed words to form a clean text
    clean_text = ' '.join(processed_words)

    return clean_text

In [4]:
def create_tables(cursor_all, cursor_stars):
    """
    Create tables in the respective SQLite databases
    Inputs: cursor_all, cursor_stars: Prefined SQLIte databases.
    Returns: None
    """
    # Create table for all_features.db
    cursor_all.execute("""
        CREATE TABLE IF NOT EXISTS data (
            review_id TEXT PRIMARY KEY,
            user_id TEXT,
            business_id TEXT,
            stars REAL,
            useful REAL,
            funny REAL,
            cool REAL,
            text TEXT,
            processed_text TEXT,
            date TEXT
        )
    """)

    # Create table for star_reviews.db
    cursor_stars.execute("""
        CREATE TABLE IF NOT EXISTS data (
            stars REAL,
            processed_text TEXT
        )
    """)

In [5]:
def insert_data(cursor_all, cursor_stars, review):
    """
    Inserts a single review into the SQLite databases
    Inputs: cursor_all, cursor_stars: the SQLIte databases, containing all features, and just stars/reviews
            review: the current JSON file line being procesed and inserted
    Returns: None, but appends line to SQLIte databses
    """
    review_id = review.get('review_id')
    user_id = review.get('user_id')
    business_id = review.get('business_id')
    stars = review.get('stars')
    stars = float(stars)

    useful = review.get('useful')
    funny = review.get('funny')
    cool = review.get('cool')
    text = review.get('text', '')
    processed_text = text_preprocess(text)
    date = review.get('date')

    # Insert into all_features.db
    cursor_all.execute("""
        INSERT OR IGNORE INTO data 
        (review_id, user_id, business_id, stars, useful, funny, cool, text, processed_text, date)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (review_id, user_id, business_id, stars, useful, funny, cool, text, processed_text, date))

    # Insert into star_reviews.db
    cursor_stars.execute("""
        INSERT OR IGNORE INTO data 
        (stars, processed_text)
        VALUES (?, ?)
    """, (stars, processed_text))


In [6]:
# Define input file name and Sqlite databses
input_file = "yelp_academic_dataset_review.json"
all_data_db = "all_features.db"
star_reviews_db = "star_reviews.db"

# Connect to SQLite databases
conn_all = sqlite3.connect(all_data_db)
conn_stars = sqlite3.connect(star_reviews_db)

cursor_all = conn_all.cursor()
cursor_stars = conn_stars.cursor()

# Create tables
create_tables(cursor_all, cursor_stars)

 # Open and parse the JSON file using ijson with multiple_values=True
with open(input_file, 'r', encoding='utf-8') as f:
    # Initialize ijson parser for multiple top-level JSON objects
    objects = ijson.items(f, '', multiple_values=True)
    batch_size = 1500
    count = 0
    for review in objects:
        if isinstance(review, dict):
            insert_data(cursor_all, cursor_stars, review)
            count += 1

            # Commit every batch_size records
            if count % batch_size == 0:
                conn_all.commit()
                conn_stars.commit()
                print(f"Inserted {count} records.")


# Commit all transactions
conn_all.commit()
conn_stars.commit()

# Close database connections
conn_all.close()
conn_stars.close()

print("Data insertion complete.")


Inserted 1500 records.
Inserted 3000 records.
Inserted 4500 records.
Inserted 6000 records.
Inserted 7500 records.
Inserted 9000 records.
Inserted 10500 records.
Inserted 12000 records.
Inserted 13500 records.


KeyboardInterrupt: 